In [17]:
# Imports
from iexfinance.stocks import Stock
import requests
import urllib.parse
import requests
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import time
import seaborn as sns

In [63]:
# params
period = 'quarter'
time = '24'
symbol = ["ADBE","CRM"]
token = 'Tsk_910a8ace8ad14af698bc4a65be1a4219'
portfolio = {}

In [64]:
# get df from api

def get_df(ls, period, time, token):
    for ticker in ls:
        financials_url = f"https://sandbox.iexapis.com/stable/stock/{ticker}/financials?period={period}&last={time}&token={token}"
        financials_json = requests.get(financials_url).json()
        portfolio[ticker] = pd.DataFrame(financials_json['financials'])

In [65]:
# preprocessing

def preprocess(dic):
    for v in dic.values():
        v.rename(str.lower, axis='columns', inplace=True)
        v.set_index(pd.to_datetime(v['fiscaldate'].apply(lambda x: x[:7])), inplace=True)

In [66]:
# add columns

def add_profitability(dic):
    for df in dic.values():
        df['operating_margin'] = df['operatingincome'] / df['revenue']
        df['net_margin'] = df['netincome'] / df['revenue']
        df['asset_turnover'] = df['revenue'] / df['totalassets']
        df['roa'] = df['netincome']*4/ df['totalassets']
        df['equity_multipl'] = df['totalassets'] / df['shareholderequity']
        df['roe'] = df['netincome'] / df['shareholderequity']
        #df['fcf_margin'] = df['fcf'] / df['revenue']

def add_financial_strength(dic):
    for df in dic.values():
        df['cash_debt'] = df['totalcash'] / df['totaldebt']
        df['equity_asset'] = df['shareholderequity'] / df['totalassets']
        df['debt_equity'] = df['totaldebt'] / df['shareholderequity']
        df['debt_ebitda'] = df['totaldebt'] / (df['ebitda']*4)